In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-core

In [ ]:
!pip install langchain-community

In [ ]:
!pip install -qU \
    langchain==0.2.0 \
    anthropic==0.7.7 \
    cohere==4.37 \
    docarray==0.39.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.1/808.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install python-dotenv


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("your_api_key")

# This is the YouTube video we're going to use.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=YwmZ6AUPU_w"

In [ ]:
from langchain.chat_models import ChatAnthropic

ANTHROPIC_API_KEY = "your_api_key"

model = ChatAnthropic(
    model="claude-2.1",
    max_tokens_to_sample=512,
    anthropic_api_key=ANTHROPIC_API_KEY
)  # swap Anthropic for OpenAI with `ChatOpenAI` and `openai_api_key`

##Test the model by asking a simple question:

In [ ]:
model.invoke("Name 3 yellow spring flowers.")

"""AIMessage(content='1. Daffodil\n2. Forsythia\n3. Tulip', response_metadata={'finish_reason': 'stop', 'logprobs': None})"""

"AIMessage(content='1. Daffodil\n2. Forsythia\n3. Tulip', response_metadata={'finish_reason': 'stop', 'logprobs': None})"

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Name 3 yellow spring flowers.")

"""1. Daffodil
2. Forsythia
3. Tulip"""

'1. Daffodil\n2. Forsythia\n3. Tulip'

##Introducing prompt templates

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Varsha lives in Bangalore", question="Where does Varsha live?")

'Human: \nAnswer the question based on the context below. If you can\'t\nanswer the question, reply "I don\'t know".\n\nContext: Varsha lives in Bangalore\n\nQuestion: Where does Varsha live?\n'

##chain the prompt with the model and the output parser:

In [ ]:
chain = prompt | model | parser
chain.invoke({
    "context": "Varsha lives in Bangalore",
    "question": "Where does Varsha live?"
})

"""Varsha lives in Bangalore."""

'Varsha lives in Bangalore.'

##Combining chains

In [ ]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [ ]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Varsha likes Ice-cream. Varsha's brother name is Badal.",
        "question": "How many siblings does Varsha have?",
        "language": "Spanish",
    }
)

"""Varsha tiene un hermano, su hermano Badal."""

'Varsha tiene un hermano, su hermano Badal.'

##Transcribing the YouTube Video
Let’s download the video and transcribe it using OpenAI’s Whisper.

In [ ]:
! pip install openai-whisper
!pip install pytube

import tempfile
import whisper
from pytube import YouTube


# Let's do this only if we haven't created the transcription file yet.
if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()

    # Let's load the base model. This is not the most accurate
    # model but it's fast.
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("transcription.txt", "w") as file:
            file.write(transcription)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cac

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 137MiB/s]


##Let’s read the transcription and display the first few characters to ensure everything works as expected.

In [ ]:
with open("transcription.txt") as file:
    transcription = file.read()

transcription[:100]



"Hello everyone, today we are going to see a speech on importance of education. So let's start the vi"

##Using the entire transcription as context

In [ ]:
try:
    chain.invoke({
        "context": transcription,
        "question": "what is Role of  education ?"
    })
except Exception as e:
    print(e)



##Splitting the transcription

In [ ]:
!pip install langchain_core

In [ ]:
import langchain_core

print(langchain_core.__version__)



0.2.0


In [ ]:
!pip install --upgrade langchain_core

In [ ]:
from langchain_core.runnables import run_in_executor

print(run_in_executor)

<function run_in_executor at 0x7c69c4438d30>


In [ ]:
!pip install langchain_community

from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcription.txt")
text_documents = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

## Generate embeddings

In [ ]:
#from langchain_openai.embeddings import OpenAIEmbeddings

#embeddings = OpenAIEmbeddings(openai_api_key="API_KEY")

In [ ]:
!pip install langchain_cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: cohere
    Found existing installation: cohere 4.37
    Uninstalling cohere-4.37:
      Successfully uninstalled cohere-4.37


In [ ]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(cohere_api_key="your_api_key")

In [ ]:
import os

# Set the Pinecone API key in the environment variable
os.environ["PINECONE_API_KEY"] = "your_api_key"

In [ ]:
!pip install langchain_pinecone
!pip install pinecone-client

from pinecone import Pinecone
pc = Pinecone(api_key=os.environ.get("api_key"))

from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough

index_name = "youtube-index"

# connect to index
index = pc.Index(index_name)

# view index stats
index.describe_index_stats()

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

In [ ]:
pinecone.similarity_search("What is benefits of education ?")[:3]

[Document(page_content='good education becomes good citizen of the country. A country can progress well if its citizens are well educated. All in all, education is the process of acquiring knowledge which helps to be successful. There are many benefits of good education like having a good career, improved self-confidence and having a good status in society. Education helps to grow in life. It is the reason behind many successful people and nations. Above all, it helps in building a better society for the citizens of a nation. It helps to eliminate the darkness of ignorance and bring light to the world. Thank you. Thanks for watching have a nice day.', metadata={'source': 'transcription.txt'}),
 Document(page_content="Hello everyone, today we are going to see a speech on importance of education. So let's start the video. Good morning to one and all present here. Today I am here to share my views about importance of education. Education plays a very important role in one's life. It helps

##Let’s setup the new chain using Pinecone as the vector store:

In [ ]:
chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

chain.invoke("What is benefits of education?")



' Based on the context, some of the benefits of education mentioned are:\n\n1) Having a good career\n2) Improved self-confidence \n3) Having a good status in society\n4) Helps to be successful\n5) Helps in building a better society\n6) Helps to eliminate the darkness of ignorance\n7) Improves our knowledge, skills, confidence level and personality\n8) Empowers us intellectually \n9) Helps us to deal with problems\n10) Helps in getting a good job and position\n11) Prepares people for different life experiences\n12) Polishes our mind and strengthens our character\n13) Helps us to take right decisions\n14) Key to success in personal and professional life\n15) Makes us smart enough to handle tough situations \n16) Helps us to live a happy life'